In [112]:
# Libraries
from fpdf import FPDF
import pandas as pd
import yaml


In [113]:
# Define the path and filename
yaml_path = "./page_layout.yaml"


# Load the YAML file
with open(yaml_path , 'r') as file:
    data = yaml.safe_load(file)
    
title_report = "US Propane"


In [114]:
class report_pages:
    def __init__(self, header_img_path, chart_img_paths, output_file):
        self.pdf = FPDF()
        self.header_img_path = header_img_path
        self.chart_img_paths = chart_img_paths
        self.output_file = output_file
    
    def add_singular_chart(self, num_pos):
        """Add the chart images to the PDF"""
        chart_image_path = self.chart_img_paths[num_pos]
        y_position = self.pdf.get_y() + 55  # Update the y_position after adding the image
        page_width = self.pdf.w
        image_width = 200
        left_margin = (page_width - image_width) / 2  # Center the image
        self.pdf.image(chart_image_path, x=left_margin, y=y_position, w=image_width)
        self.pdf.ln(90)
        
    def add_charts(self, positions_to_include):
        """
        Add the chart images to the PDF in a 2x2 grid layout using specified positions.

        Parameters:
            positions_to_include (list): List of indices (0-based) for chart_img_paths to include.
        """
        page_width = self.pdf.w
        page_height = self.pdf.h
        image_width = 97  # Image width (adjust to fit layout)
        image_height = 74  # Image height (adjust to fit layout)
        horizontal_margin = 5  # Space between images horizontally
        vertical_margin = -3  # Space between images vertically

        # Initial y-position (can be set to any desired value)
        initial_y_position = 150  # Start a bit lower on the page (e.g., 50 units from top)
        current_y_position = initial_y_position

        # Define the grid positions
        positions = [
            [horizontal_margin, current_y_position],  # Top-left
            [horizontal_margin + image_width + horizontal_margin, current_y_position],  # Top-right
            [horizontal_margin, current_y_position + image_height + vertical_margin],  # Bottom-left
            [horizontal_margin + image_width + horizontal_margin, current_y_position + image_height + vertical_margin],  # Bottom-right
        ]

        # Get the selected chart image paths based on positions_to_include
        selected_chart_img_paths = [self.chart_img_paths[i] for i in positions_to_include]

        # Iterate over the selected chart image paths and place them in the grid
        for i, chart_image_path in enumerate(selected_chart_img_paths):
            if i >= 4:  # Stop after 4 images to fit the 2x2 grid
                break
            
            x_position, y_position = positions[i]
            
            # Add image to the PDF at the calculated position
            self.pdf.image(chart_image_path, x=x_position, y=y_position, w=image_width, h=image_height)

        # Move the cursor down after images are placed
        self.pdf.ln(image_height + vertical_margin)


    def add_header(self):
        """Add the header image and title to the PDF"""
        self.pdf.add_page()
        self.pdf.image(self.header_img_path, x=0, y=0, w=211)
        self.pdf.set_text_color(255, 255, 255)  # RGB for black
        self.pdf.set_xy(1, 14)
        self.pdf.add_font('Montserrat-Bold', 'B', './assets/Montserrat/static/Montserrat-Bold.ttf', uni=True)
        self.pdf.set_font('Montserrat-Bold', 'B', 24)
        self.pdf.multi_cell(0, 10, f"{title_report}", 0, 'L')
        
    def generate_report(self, title_report):
        """Generate and save the report as a PDF"""
        # Page 1
        self.add_header()
        self.add_singular_chart(0)
        self.add_charts([1,2,3,4])
        # self.add_header()
        self.pdf.output(self.output_file)

In [115]:
# Usage example:
header_img_path = './assets/header.png'
chart_img_paths = [
    '../report/images/us_propane_stocks.png',
    '../report/images/padd1_propane_stocks.png',
    '../report/images/padd2_propane_stocks.png',
    '../report/images/padd3_propane_stocks.png',
    '../report/images/padd4n5_propane_stocks.png',
]

# Get today's date in the desired format
from datetime import datetime
today_date = datetime.now().strftime("%Y %m %d")

# Generate the output file path with the formatted date
output_file = f'../report/{title_report} - {today_date}.pdf'

report = report_pages(header_img_path, chart_img_paths, output_file)
report.generate_report(title_report)